In [ ]:
with open('Arxiv-10k-1536D-ada2', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
# Load the data file
test =[]
with open('arxiv-ada002-1536-10k.json', 'r') as d:
    test = json.load(d)

In [ ]:
test[0]

{'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\\npresented for the production of massive photon pairs at hadron colliders. All\\nnext-to-leading order perturbative contributions from quark-antiquark,\\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\\nall-orders resummation of initial-state gluon radiation valid at\\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\\nspecified in which the calculation is most reliable. Good agreement is\\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\\nmore detailed tests with CDF and DO data. Predictions are shown for\\ndistributions of diphoton pairs produced at the energy of the Large Hadron\\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\\nboson are contrasted with those produced from QCD processes at the LHC, showing\\nthat enhanced sensitivity to the signal can be obtained with judicious\

In [ ]:
# Load the data file
data =[]
with open('arxiv-ada002-1536-10k.json', 'r') as d:
    data = json.load(d)

In [ ]:
import ast
ast.literal_eval(data[0]['authors_parsed'])

[['Balázs', 'C.', ''],
 ['Berger', 'E. L.', ''],
 ['Nadolsky', 'P. M.', ''],
 ['Yuan', 'C. -P.', '']]

In [ ]:
" ".join(data[0]['authors_parsed'])

'[ [ " B a l \\ u 0 0 e 1 z s " ,   " C . " ,   " " ] ,   [ " B e r g e r " ,   " E .   L . " ,   " " ] ,   [ " N a d o l s k y " ,   " P .   M . " ,   " " ] ,   [ " Y u a n " ,   " C .   - P . " ,   " " ] ]'

In [ ]:
for i in range(len(data)):
    data[i]['authors_parsed'] = ast.literal_eval(data[i]['authors_parsed'])
    data[i]['versions'] = ast.literal_eval(data[i]['versions'])

In [ ]:

data[0]

{'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\\npresented for the production of massive photon pairs at hadron colliders. All\\nnext-to-leading order perturbative contributions from quark-antiquark,\\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\\nall-orders resummation of initial-state gluon radiation valid at\\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\\nspecified in which the calculation is most reliable. Good agreement is\\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\\nmore detailed tests with CDF and DO data. Predictions are shown for\\ndistributions of diphoton pairs produced at the energy of the Large Hadron\\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\\nboson are contrasted with those produced from QCD processes at the LHC, showing\\nthat enhanced sensitivity to the signal can be obtained with judicious\

In [ ]:
# Define computed properties
computed_properties = [
    {'name': "cp_abstractLower", 'query': "SELECT VALUE LOWER(c.abstract) FROM c"}, 
    {'name': "cp_abstractLength",'query': "SELECT VALUE LENGTH(c.abstract) FROM c"},
    {'name': "cp_authorsCount",'query': 'SELECT VALUE ARRAY_LENGTH(c.authorsparsed) FROM c'},
    {'name': "cp_toVectorize",'query': 'SELECT VALUE CONCAT("The paper: ", c.title, " with abstract: ", c.abstract, " is by the authors: ", c.authors) FROM c'}
    ]

vector_embedding_policy = {
    "vectorEmbeddings": [ 
        { 
            "path":"/Embedding",
            "dataType":"float32",
            "distanceFunction":"cosine",
            "dimensions": 1536
        }, 
    ]
}

indexing_policy = {
    "vectorIndexes": [ 
        {
            "path": "/Embedding", 
            "type": "diskANN" 
        }
    ],
      "includedPaths": [
        {
            "path": "/*"
        },
        {   
            "path": "/cp_abstractLower/?"
        },
        {
            "path": "/cp_abstractLength/?"
        },
        {
            "path": "/cp_toVectorize/?"
        },
        {
            "path": "/cp_authorsCount/?"
        }
      ],
      "excludedPaths": [
        {
            "path": "/\"_etag\"/?"
        }
      ]
} 

container = db.create_container_if_not_exists(id="ArxivPapers",
                                                 partition_key=PartitionKey(path='/id'),
                                                 computed_properties=computed_properties,
                                                 vector_embedding_policy=vector_embedding_policy,
                                                 indexing_policy=indexing_policy,
                                                 offer_throughput=1000)

In [ ]:
counter = 0
for item in data:
    #The following code to create vector embeddings for the data is commented out as the sample data is already vectorized.
    #vectorArray = generate_embeddings("Title:" + data[i]['original_title'] + ", Tagline:" + data[i]['tagline'] + ", Overview:" + data[i]['overview'])
    #object[cosmos_vector_property] = vectorArray
    container.upsert_item(body=item)

    # print progress every 100 upserts. 
    counter += 1
    if counter % 100 == 0:
            print("Inserted {} documents into collection.".format(counter))

Inserted 100 documents into collection.
Inserted 200 documents into collection.
Inserted 300 documents into collection.
Inserted 400 documents into collection.


KeyboardInterrupt: 